# Extraindo dados de vagas do Linkedin através de Web Scraping

### Instalando as bibliotecas necessárias

In [1]:
!pip install requests # Usada para fazer requisições HTTP, ou seja, acessar páginas da web e pegar seu conteúdo.
!pip install beautifulsoup4 # Usada para processar e interpretar o código HTML das páginas da web.


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


### Importando as Bibliotecas

In [1]:
# Importando as bibliotecas necessárias

import requests  # Para acessar páginas da web
from bs4 import BeautifulSoup  # Para extrair informações do código HTML das páginas
import pandas as pd  # Para criar e manipular tabelas de dados

### Coletando informações de filtros para a Vaga

In [2]:
nome_vaga = input("Digite o nome da vaga: ")
nome_vaga_tratada = nome_vaga.replace(" ", "%20").lower()

In [3]:
experiencia = input("Digite o tipo de experiência: (0 = Todos, 1 = Estágio, 2 = Assistente, 3 = Júnior, 4 = Pleno-Sênior e 5 = Diretor)")
if experiencia == "0" or int(experiencia) > 5:
    experiencia_tratada = ''
else:
    experiencia_tratada = f'''&f_E={experiencia}'''

In [4]:
tipo_vaga = input("Digite o tipo da vaga: (0 = Todos, 1 = Presencial, 2 = Remoto, 3 = Híbrido)")
if tipo_vaga == "0" or int(tipo_vaga) > 3:
    tipo_vaga_tratada = ''
else:
    tipo_vaga_tratada = f'''&f_WT={tipo_vaga}'''

### Criando a URL de busca

In [5]:
base_url = f'''
https://www.linkedin.com/jobs/search?
keywords={nome_vaga_tratada}
&location=Brazil
&geoId=106057199
{experiencia_tratada}
&f_TPR=r86400
{tipo_vaga_tratada}
&position=1
&pageNum=0 
'''

In [6]:
base_url

'\nhttps://www.linkedin.com/jobs/search?\nkeywords=analista%20de%20dados\n&location=Brazil\n&geoId=106057199\n&f_E=2\n&f_TPR=r86400\n&f_WT=2\n&position=1\n&pageNum=0 \n'

In [7]:
base_url = base_url.replace("\n", "")

In [8]:
base_url

'https://www.linkedin.com/jobs/search?keywords=analista%20de%20dados&location=Brazil&geoId=106057199&f_E=2&f_TPR=r86400&f_WT=2&position=1&pageNum=0 '

### Fazendo a requisição para o Linkedin

In [9]:
response = requests.get(base_url)

In [10]:
print(response)

<Response [200]>


### Extraindo os dados da Página

In [11]:
site = BeautifulSoup(response.text, 'html.parser')

In [12]:
site

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="d_jobs_guest_search" name="pageKey"/>
<!-- --> <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
<meta content="en_US" name="locale"/>
<meta data-app-version="2.0.2288" data-browser-id="6206d551-6920-4b7d-8f9a-e3acb89ae472" data-call-tree-id="AAYuf2uMj0CemevrLd7uEQ==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="enabled" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;2sqiyTTYQ5mr0Odv4yaCog==" data-service-name="jobs-guest-frontend" data-should-use-full-url-in-pve-path="true" data-sync-apfc-cb-lix-treatment="enabled" data-sync-apfc-headers-lix-treatment="control" id="config"/>
<link href="https://br.linkedin.com/jobs/analista-de-dados-vagas" rel="canonical"/>
<!-- --><!-- -->
<!-

### Encontrando os elementos com informações das vagas

In [13]:
dados = site.find_all("div", attrs={"class" : "base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card"})

### Criando uma lista para armazenar os dados

In [14]:
armazenando_vaga = []
armazenamento_empresa = []
armazenamento_localizacao = []
armazenamento_tempo = []
armazenamento_data = []
armazenamento_link_empresa = []
armazenamento_link_vaga = []

### Extraindo informações de cada Vaga

In [17]:
for palavra in dados:
    # Nome da vaga
    nome_vaga = palavra.find("h3", attrs={"class": "base-search-card__title"})
    nome_vaga = nome_vaga.text.strip() if nome_vaga else "Sem Informação"
    armazenando_vaga.append(nome_vaga)

    # Nome da empresa
    nome_empresa = palavra.find("h4", attrs={"class": "base-search-card__subtitle"})
    nome_empresa = nome_empresa.text.strip() if nome_empresa else "Sem Informação"
    armazenamento_empresa.append(nome_empresa)

    # Localização
    localizacao = palavra.find("span", attrs={"class": "job-search-card__location"})
    localizacao = localizacao.text.strip() if localizacao else "Sem Informação"
    armazenamento_localizacao.append(localizacao)

    # Tempo de disponibilidade da vaga
    tempo_vaga = palavra.find("time")
    tempo_vaga = tempo_vaga.text.strip() if tempo_vaga else "Sem Informação"
    armazenamento_tempo.append(tempo_vaga)

    # Data
    data = palavra.find("time", attrs={"class": "job-search-card__listdate--new"})
    data_trat = data["datetime"] if data else "Sem Informação"
    armazenamento_data.append(data_trat)

    # Link da vaga
    link_vaga = palavra.find("a", attrs={"class": "base-card__full-link"})
    link_vaga = link_vaga.get("href") if link_vaga else "Sem Link"
    armazenamento_link_vaga.append(link_vaga)

    # Link da empresa
    link_empresa = palavra.find("a", attrs={"class": "hidden-nested-link"})
    link_empresa = link_empresa.get("href") if link_vaga else "Sem Link"
    armazenamento_link_empresa.append(link_vaga)

print(f"Total de vagas armazenadas: {len(armazenando_vaga)}")


Total de vagas armazenadas: 168


### Criando um dicionário com os dados coletados


In [18]:
base_vagas = {
"vaga" : armazenando_vaga,
"empresa": armazenamento_empresa,
"localizacao" : armazenamento_localizacao,
"Tempo_postada": armazenamento_tempo,
"data_vaga" : armazenamento_data,
"link_vaga" : armazenamento_link_vaga,
"link_empresa" : armazenamento_link_empresa
}

### Criando um dataframe para armazenar os dados

In [19]:
df = pd.DataFrame(base_vagas)

In [20]:
df

,vaga,empresa,localizacao,Tempo_postada,data_vaga,link_vaga,link_empresa
0,Analista de Dados Pleno - Comercial,O Novo Mercado,"Santos, São Paulo, Brazil",19 hours ago,2025-02-18,https://br.linkedin.com/jobs/view/analista-de-...,https://br.linkedin.com/jobs/view/analista-de-...
1,Analista de Dados Júnior (Google e Facebook Ads),Culture Designer,"Joinville, Santa Catarina, Brazil",9 hours ago,2025-02-19,https://br.linkedin.com/jobs/view/analista-de-...,https://br.linkedin.com/jobs/view/analista-de-...
2,Coletor de Dados,K-Lab,Greater São Paulo Area,10 hours ago,2025-02-19,https://br.linkedin.com/jobs/view/coletor-de-d...,https://br.linkedin.com/jobs/view/coletor-de-d...
3,Data Analyst / Business Data Analyst,Sunco.com,Brazil,1 hour ago,2025-02-19,https://br.linkedin.com/jobs/view/data-analyst...,https://br.linkedin.com/jobs/view/data-analyst...
4,Analista de CRM Junior,XP Inc.,"São Paulo, São Paulo, Brazil",2 hours ago,2025-02-19,https://br.linkedin.com/jobs/view/analista-de-...,https://br.linkedin.com/jobs/view/analista-de-...
...,...,...,...,...,...,...,...
163,Engenheiro de software net,Netvagas,Brazil,19 hours ago,2025-02-18,https://br.linkedin.com/jobs/view/engenheiro-d...,https://br.linkedin.com/jobs/view/engenheiro-d...
164,Desenvolvedor(a) Mainframe SR,Solutis Tecnologias,Brazil,15 hours ago,2025-02-18,https://br.linkedin.com/jobs/view/desenvolvedo...,https://br.linkedin.com/jobs/view/desenvolvedo...
165,Desenvolvedor a front end pleno,Netvagas,Brazil,1 hour ago,2025-02-19,https://br.linkedin.com/jobs/view/desenvolvedo...,https://br.linkedin.com/jobs/view/desenvolvedo...
166,.NET Engineer (Desktop app),Mimica,Brazil,14 hours ago,2025-02-18,https://br.linkedin.com/jobs/view/net-engineer...,https://br.linkedin.com/jobs/view/net-engineer...


### Proximos passos, Fazer a inclusão dos daos em um banco de dados e subir para a AWS

In [21]:
import sqlite3
import pandas as pd
from datetime import datetime

# Criar a conexao com meu banco de dados SQLITE
nome_banco = "vagas_linkedin2.db"
conector = sqlite3.connect(nome_banco)
cursor = conector.cursor()

# criando nossa tabela para armazenar as vagas

cursor.execute('''
CREATE TABLE IF NOT EXISTS vagas(
id INTEGER PRIMARY KEY AUTOINCREMENT,
vaga TEXT NOT NULL,
empresa TEXT NOT NULL,
localizacao TEXT NOT NULL,
Tempo_postada TEXT NOT NULL,
data_vaga DATE NOT NULL,
link_vaga TEXT UNIQUE NOT NULL,
link_empresa TEXT NOT NULL,
data_insercao DATE NOT NULL
)
''')

In [22]:
# Adicionando data no Python
data_hoje = datetime.now().strftime("%Y-%m-%d")
df["data_insercao"] = data_hoje

dados_linkedin = list(df.itertuples(index = False, name = None))

cursor.executemany('''
  INSERT OR IGNORE INTO vagas (vaga, empresa, localizacao, Tempo_postada, data_vaga, link_vaga, link_empresa, data_insercao)
  VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', dados_linkedin)

# salvar as infos no banco
conector.commit()

# fechar conexao
conector.close()